## Post-process wflow_sbm output

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from glob import glob
from pathlib import Path

import os
import iris
import xarray as xr
import pandas as pd
from esmvalcore.preprocessor import regrid
from pathos.threading import ThreadPool as Pool
from dask.diagnostics import ProgressBar

## Set Paths

In [3]:
# Set Paths
ROOT = Path('/gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/')
MODELS = Path(f'{ROOT}/model_parameters/')
AUXDIR = Path(f'{ROOT}/aux_data/')
RESULTS = Path(f"{ROOT}/results/streamflow_evaluation/wflow_sbm_calibration/")
OUTPUT = Path(f'{ROOT}/results/')

## Config

In [4]:
# Get available basin IDs wflow_sbm
calibration_file = f"{RESULTS}/wflow_calibration_objective_function_overview.csv"
df = pd.read_csv(calibration_file, index_col='basin_id')
basin_ids = df.index.to_list()

# Amount of available cores
cores_available = 1

## Post-process wflow_sbm soil moisture (VWC)

In [7]:
def post_sm_output(basin_id):
    print(basin_id)
    # Set basin directory
    BASINDIR = f'{MODELS}/wflow_sbm/{basin_id}/'

    # Load data
    ds_sim = xr.open_dataset(f'{BASINDIR}/evaluation/output.nc')

    # Get soil moisture data
    da_sm = ds_sim.vwc.sel(layer=1)

    # Subset time
    da_sm = da_sm.sel(time=slice('2015','2017'))

    # Calculate time series
    da_sm = da_sm.mean(['lat','lon'])
    da_sm = da_sm.chunk(chunks='auto')
    da_sm = da_sm.drop('layer')

    # Output filename
    output_fname = f'{OUTPUT}/soil_moisture_evaluation/regridded_wflow_sbm_sm/{basin_id}_wflow_sbm_sm_2015_2017.nc'

    # Save to netcdf
    write_job = da_sm.to_netcdf(output_fname, compute=False)
    with ProgressBar():
        write_job.compute()

    return print(f'{basin_id} finished: {output_fname}')

## Parallel run function

In [8]:
def parallel_run_sm(
    basin_ids,
    threads=cores_available,
    ):
    
    # Set number of threads (cores) used for parallel run and map threads
    if threads is None:
        pool = Pool()
    else:
        pool = Pool(nodes=threads)
    # Run parallel models
    pool.map(
        post_sm_output,
        basin_ids,
        )
    return

## Sort basins by size

In [9]:
# Sort by basin size
def sort_basin_ids_by_size(basin_ids):
    sizes = []
    for basin_id in basin_ids:
        size = os.path.getsize(f'{MODELS}/wflow_sbm/{basin_id}/staticmaps.nc')
        sizes.append(size)

    df = pd.DataFrame()
    df['basin_id'] = basin_ids
    df['size'] = sizes
    df = df.sort_values(by='size',ascending=False)

    basin_ids = df.basin_id.to_list()
    
    return basin_ids

basin_ids_sorted = sort_basin_ids_by_size(basin_ids)

## Run parallel function

In [10]:
# Run function
# parallel_run_sm(basin_ids_sorted)
parallel_run_sm(basin_ids)

10003
[########################################] | 100% Completed |  0.1s
10003 finished: /gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/results/soil_moisture_evaluation/regridded_wflow_sbm_sm/10003_wflow_sbm_sm_2015_2017.nc
1001
[########################################] | 100% Completed |  0.1s
1001 finished: /gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/results/soil_moisture_evaluation/regridded_wflow_sbm_sm/1001_wflow_sbm_sm_2015_2017.nc
101002
[########################################] | 100% Completed |  0.1s
101002 finished: /gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/results/soil_moisture_evaluation/regridded_wflow_sbm_sm/101002_wflow_sbm_sm_2015_2017.nc
101005
[########################################] | 100% Completed |  0.1s
101005 finished: /gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/results/soil_moisture_evaluation/regridded_wflow_sbm_sm/101005_wflow_sbm_sm_2015_2017.nc
102001
[########################################] | 100% 

## Post-process wflow_sbm evaporation

In [7]:
def post_et_output(basin_id):
    print(basin_id)
    # Set basin directory
    BASINDIR = f'{MODELS}/wflow_sbm/{basin_id}/'

    # Load data
    ds_sim = xr.open_dataset(f'{BASINDIR}/evaluation/output.nc')

    # Get evaporation data
    da_et = ds_sim.actualevap + ds_sim.interception

    # Subset time
    da_et = da_et.sel(time=slice('2008','2015'))

    # Calculate time series
    da_et = da_et.mean(['lat','lon'])
    da_et = da_et.chunk(chunks='auto')
    
    # Resample to 8 daily interval per year
    sim_years = []
    years = da_et.resample(time='Y').mean().time.dt.year
    for year in years.values:
        ds_sim_year = da_et.sel(time=str(year))
        sim_years.append(ds_sim_year.resample(time='8D').mean())

    da_et = xr.concat(sim_years, dim='time')
    
    
    # Output filename
    output_fname = f'{OUTPUT}/evaporation_evaluation/regridded_wflow_sbm_evaporation/{basin_id}_wflow_sbm_et_2008_2015.nc'

    # Save to netcdf
    write_job = da_et.to_netcdf(output_fname, compute=False)
    with ProgressBar():
        write_job.compute()

    return print(f'{basin_id} finished: {output_fname}')

## Parallel run function

In [8]:
def parallel_run_et(
    basin_ids,
    threads=cores_available,
    ):
    
    # Set number of threads (cores) used for parallel run and map threads
    if threads is None:
        pool = Pool()
    else:
        pool = Pool(nodes=threads)
    # Run parallel models
    pool.map(
        post_et_output,
        basin_ids,
        )
    return

## Sort basins by size

In [9]:
# Sort by basin size
def sort_basin_ids_by_size(basin_ids):
    sizes = []
    for basin_id in basin_ids:
        size = os.path.getsize(f'{MODELS}/wflow_sbm/{basin_id}/staticmaps.nc')
        sizes.append(size)

    df = pd.DataFrame()
    df['basin_id'] = basin_ids
    df['size'] = sizes
    df = df.sort_values(by='size',ascending=False)

    basin_ids = df.basin_id.to_list()
    
    return basin_ids

basin_ids_sorted = sort_basin_ids_by_size(basin_ids)

## Run parallel function

In [9]:
# Run function
parallel_run_et(basin_ids_sorted)

[########################################] | 100% Completed |  0.4s
54057 finished: /gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/results/evaporation_evaluation/regridded_wflow_sbm_evaporation/54057_wflow_sbm_et_2008_2015.nc
[########################################] | 100% Completed |  0.3s
33035 finished: /gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/results/evaporation_evaluation/regridded_wflow_sbm_evaporation/33035_wflow_sbm_et_2008_2015.nc
[########################################] | 100% Completed |  0.3s
18010 finished: /gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/results/evaporation_evaluation/regridded_wflow_sbm_evaporation/18010_wflow_sbm_et_2008_2015.nc
[########################################] | 100% Completed |  0.3s
41014 finished: /gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/results/evaporation_evaluation/regridded_wflow_sbm_evaporation/41014_wflow_sbm_et_2008_2015.nc
[########################################] | 100% Comple

IndexError: index -1 is out of bounds for axis 0 with size 0